In this step we are filtering the tweets of the Key Opinion Leaders, that we retrived in the first step. 


In [1]:
from twitter_crawler import TwitterCrawler

my_token = 'AAAAAAAAAAAAAAAAAAAAADRFOwEAAAAAaTp%2Bdd1OhobYMYb5ExPXm7IL6RA%3DDHknH402gOXoegUGxNtpC6giIjdackRLtdRx6tjrnnLeFN1ntT'
my_twitter_crawler = TwitterCrawler(bearer_token= my_token)


In [2]:
import pandas as pd
### note that tweets_table_filtered_file_name file needs to be in xlsx format
def simple_preprocess_on_tweets_table_filtered (tweets_table_filtered_file_name = "tweets_table_filtered.xlsx"):
    tweets_table_filtered = pd.read_excel(tweets_table_filtered_file_name)
    tweets_table_filtered['conversation_id'] = tweets_table_filtered['conv_id_new'].apply(lambda x: x.replace("conv_id: ", ""))
    tweets_table_filtered = tweets_table_filtered.add_prefix('KOL_Tweet_')

    return tweets_table_filtered



In [6]:
### reading the filtered tweets table
tweets_table_filtered = simple_preprocess_on_tweets_table_filtered("tweets_table_filtered_final_by_rawwaa.xlsx")
print("There are",tweets_table_filtered.shape[0], "Filtered tweets")
display(tweets_table_filtered.head())


There are 1513 Filtered tweets


,KOL_Tweet_Unnamed: 0,KOL_Tweet_Unnamed: 0.1,KOL_Tweet_Unnamed: 0.1.1,KOL_Tweet_author_id,KOL_Tweet_author_id_new,KOL_Tweet_conv_id_new,KOL_Tweet_conversation_id,KOL_Tweet_created_at,KOL_Tweet_entities.mentions,KOL_Tweet_id,...,KOL_Tweet_Name,KOL_Tweet_Born_in,KOL_Tweet_twitter_user_name,KOL_Tweet_Role,KOL_Tweet_Place,KOL_Tweet_clean_user_name,KOL_Tweet_score_key_words,KOL_Tweet_score_KOP,KOL_Tweet_total_score,KOL_Tweet_language
0,0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,7.401591e+17,...,David DAVIS,England,@DavidDavisMP,Secretary of State for Exiting the,European Union UK Government,DavidDavisMP,4,1,25,en
1,1,24792,451,155507136,author_id: 155507136,conv_id: 953958453554401283,953958453554401283,2018-01-18T11:53:13.000Z,"[{'start': 28, 'end': 41, 'username': 'GOettin...",9.539585e+17,...,Mark RUTTE,Netherlands,@MinPres,Dutch Government,Prime Minister,MinPres,2,1,20,nl
2,2,3894,26,3131144855,author_id: 3131144855,conv_id: 1069285577869205506,1069285577869205506,2018-12-02T17:47:45.000Z,NaN,1.069287e+18,...,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,3,1,20,en
3,3,7496,22,540274837,author_id: 540274837,conv_id: 821274533080551424,821274533080551424,2017-01-17T08:39:25.000Z,NaN,8.212757e+17,...,Frances O'GRADY,England,@FrancesOGrady,General Secretary,Trades Union Congress,FrancesOGrady,2,1,20,en
4,4,24385,44,155507136,author_id: 155507136,conv_id: 1066662602443538432,1066662602443538432,2018-11-25T11:59:16.000Z,NaN,1.066663e+18,...,Mark RUTTE,Netherlands,@MinPres,Dutch Government,Prime Minister,MinPres,3,1,20,nl


In [18]:
#top_n_tweets = list(tweets_table_filtered.head()['KOL_Tweet_conversation_id'])
#top_n_tweets.append("702594185942188033")
#print(top_n_tweets)

['740159086667411456', '953958453554401283', '1069285577869205506', '821274533080551424', '1066662602443538432', '702594185942188033']


In [8]:
### saving in a variable all_conversation_ids all the conversation ids of the filtered tweets
all_conversation_ids = tweets_table_filtered['KOL_Tweet_conversation_id']
#sanity check:
print("Number of conversation ids we wish to get their comments:", len(all_conversation_ids))

Number of conversation ids we wish to get their comments: 1513


## Comments

In the following chunk we will check - wheter we already have a csv file for a given conversation id that is in the list,
If it exists there, then the code in the chunk will remove that conversation id from the final list we are passing the reriving comments function.
This is important as if we will change the filter criterions and get more / less tweets, we don't want to retrieve the comments of conversation ids we already have!

In [13]:
all_conversation_ids = all_conversation_ids #top_n_tweets #[]
max_results = 100
limit_amount_of_returned_comments = 1000000
start_time = "2014-01-01T00:00:00Z"
dir_tree_name = "Step 3 Comments"
##############################################################################################3
#### we don't want to get comments for conversation ids that we already searched for their comments!
# thus we first need to see which of the conversation_ids that are inserted in the list are **not** in the files we already have:
from os import listdir
from os.path import isfile, join

import os.path
try:
    os.mkdir(dir_tree_name)
    print("creating directory", dir_tree_name, "to insert all the tables of all the comments of all the given conversation ids")
except:
    print("The dir", dir_tree_name ,"already exist")

print()
conv_ids_we_have_data = [f.split("_")[-1] for f in listdir(dir_tree_name) if isfile(join(dir_tree_name, f, f.split("_")[-1] + "_comments.csv"))]

conv_ids_we_dont_have_data = list(set(all_conversation_ids) - set(conv_ids_we_have_data))

if len(conv_ids_we_dont_have_data) > 20:
    print("There are", len(conv_ids_we_dont_have_data), "Conversation ids to search their comments; see here the first 20:")
    print(conv_ids_we_dont_have_data[0:20])
else:
    print("Conversation ids to search their comments:",conv_ids_we_dont_have_data)
    print("-----------------------------------------------")
print()
if len(conv_ids_we_have_data) > 20:
    print("There are", len(conv_ids_we_have_data), "Conversation ids we already have data on --> here are the first 20:")
    print(conv_ids_we_have_data[0:20])
else : print("Conversation ids we already have data on:",list(set(all_conversation_ids).intersection(conv_ids_we_have_data)))


The dir Step 3 Comments already exist

There are 1350 Conversation ids to search their comments; see here the first 20:
['587941845780553728', '876344395548766208', '1068273328681623559', '823844616571064320', '973950772009426944', '1074243821351985152', '1022227276753252352', '426647985280143360', '872711752907186176', '939136458274820096', '699633650703400961', '578949893722677248', '722343456073523200', '865228517419646976', '847527876576653312', '664053362808266754', '858283451430178816', '746347158819966976', '872756911246123008', '1011608135570788352']

Conversation ids we already have data on: ['821274533080551424', '1069285577869205506', '953958453554401283', '702594185942188033', '740159086667411456', '1066662602443538432']


In [46]:
### This is the function that enables retirving the comments of the conversation ids
# note that we inserted the function the variable 'conv_ids_we_dont_have_data' so we will only search comments of conversation ids that
# we don't have (that we haven't searched for their commetns so far) 
path_for_table_dict = my_twitter_crawler.return_comments_by_tweet_ids(
                            conversation_ids = conv_ids_we_dont_have_data,
                            query = "",
                            start_time = start_time,
                            end_time = "today",
                            max_results = max_results, evaluate_last_token = False,
                            limit_amount_of_returned_comments = limit_amount_of_returned_comments,
                            verbose = False, dir_tree_name = dir_tree_name)

Bringing comments of 821274533080551424
The dir Step 3 Comments already exist
creating directory Step 3 Comments\conv_tree_for_821274533080551424 to insert all the comments of the given conversation-id
creating directory Step 3 Comments\conv_tree_for_821274533080551424\log_comments_for_conversation_id_821274533080551424 to insert all the logs of the comments for the tweet id -  821274533080551424
1 Got from twitter 100 comments, and there are more comments of that conversation-id to get, I am bringing more comments!

token to insert: 1jzu9lk96gu5npw3jm2iyp9i9bf73fd6n0qe6psek7zx
2 Got from twitter 74 comments, and there are more comments of that conversation-id to get, I am bringing more comments!

token to insert: 1jzu9lk96gu5npw3jm2iug463ouz8r0xwglmga16xxml
3 Got from twitter 86 comments, and there are more comments of that conversation-id to get, I am bringing more comments!

token to insert: 1jzu9lk96gu5npw3j40idlngr8y29xbkpukae9b1zrel
no more comments from this conversation id
Tota

In [14]:
#TwitterCrawler.get_url_by_tweet_id(tweet_id = "1072239308554362881")

In [48]:
import os
 
directory = 'Step 3 Comments'

all_comments_df_list = []
for root, dirs, files in os.walk(directory):
    for filename in files:
        if not filename.endswith('csv'): continue
        file_path = os.path.join(root, filename)
        comments_table =  pd.read_csv(file_path)
        all_comments_df_list.append( comments_table)

all_comments_df = pd.concat(all_comments_df_list)
all_comments_df['conversation_id'] = all_comments_df['conversation_id'].apply(lambda x : str(x))
all_comments_df = all_comments_df.add_prefix('Comment_')

In [49]:

all_comments_df

,Comment_Unnamed: 0,Comment_author_id,Comment_author_id_new,Comment_conv_id_new,Comment_conversation_id,Comment_created_at,Comment_entities.mentions,Comment_id,Comment_id_new,Comment_in_reply_to_user_id,...,Comment_users.created_at,Comment_users.description,Comment_users.id,Comment_users.name,Comment_users.public_metrics.followers_count,Comment_users.public_metrics.following_count,Comment_users.public_metrics.listed_count,Comment_users.public_metrics.tweet_count,Comment_users.username,Comment_users.verified
0,0,880612790368710656,author_id: 880612790368710656,coversation_id: 1066662602443538432,1066662602443538432,2018-12-06T11:46:40.000Z,"[{'start': 0, 'end': 16, 'username': 'Paterno1...",1070645699082182657,id: 1070645699082182657,968591062951432192,...,2017-06-30T02:23:44.000Z,NaN,880612790368710656,Deblikkert2,60,96,0,11739,Deblikkert2,False
1,1,72001883,author_id: 72001883,coversation_id: 1066662602443538432,1066662602443538432,2018-11-30T20:01:14.000Z,"[{'start': 0, 'end': 8, 'username': 'MinPres',...",1068595833069858818,id: 1068595833069858818,155507136,...,2009-09-06T09:12:48.000Z,"Piano- en dames ontstemmer. Blues, rock & Moro...",72001883,Jan Heijmans,833,1535,17,22131,Dereviled,False
2,2,85545795,author_id: 85545795,coversation_id: 1066662602443538432,1066662602443538432,2018-11-28T05:40:11.000Z,"[{'start': 0, 'end': 8, 'username': 'MinPres',...",1067654366679744512,id: 1067654366679744512,155507136,...,2009-10-27T10:57:18.000Z,"Rechtvaardigheid, kritisch, moeilijk wennende ...",85545795,Gertje,633,0,12,24029,Gertj001,False
3,3,343260979,author_id: 343260979,coversation_id: 1066662602443538432,1066662602443538432,2018-11-27T15:46:03.000Z,"[{'start': 0, 'end': 8, 'username': 'MinPres',...",1067444448664137731,id: 1067444448664137731,155507136,...,2011-07-27T08:53:25.000Z,NaN,343260979,Pieter Jansen,3,204,0,175,JPJansen01,False
4,4,238964288,author_id: 238964288,coversation_id: 1066662602443538432,1066662602443538432,2018-11-26T21:22:49.000Z,"[{'start': 0, 'end': 8, 'username': 'MinPres',...",1067166809567170560,id: 1067166809567170560,155507136,...,2011-01-16T13:18:04.000Z,Educator..,238964288,M Geusebroek,14,45,0,206,Mye1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,28,4838462057,author_id: 4838462057,coversation_id: 953958453554401283,953958453554401283,2018-01-18T11:57:21.000Z,"[{'start': 0, 'end': 8, 'username': 'MinPres',...",953959491594268672,id: 953959491594268672,155507136,...,2016-01-23T13:08:08.000Z,NaN,4838462057,Strijkplank,405,1002,0,15618,horenzienenzw,False
29,29,312302180,author_id: 312302180,coversation_id: 953958453554401283,953958453554401283,2018-01-18T11:56:14.000Z,"[{'start': 0, 'end': 8, 'username': 'MinPres',...",953959211070885888,id: 953959211070885888,155507136,...,2011-06-06T22:21:46.000Z,NaN,312302180,Ton Overeem,77,66,0,83,OvereemTon,False
30,30,895346756975824899,author_id: 895346756975824899,coversation_id: 953958453554401283,953958453554401283,2018-01-18T11:55:36.000Z,"[{'start': 0, 'end': 8, 'username': 'MinPres',...",953959053109157888,id: 953959053109157888,155507136,...,2017-08-09T18:11:16.000Z,NaN,895346756975824899,Taghyir en Islaah,65,334,1,11556,TaghyirI,False
31,31,272922376,author_id: 272922376,coversation_id: 953958453554401283,953958453554401283,2018-01-18T11:55:13.000Z,"[{'start': 0, 'end': 8, 'username': 'MinPres',...",953958957772615681,id: 953958957772615681,155507136,...,2011-03-27T13:06:14.000Z,"Ik Schreif zoals Ik Wil, Ik Tiep zoals IK Wil,...",272922376,moosje Pvv 👊,752,1327,3,5265,moosje666,False


In [50]:
## Merged Tweets with comments (each row is a tweet plus one of its comments, meaning that a tweet with 50 comments
# appears in 50 rows of the table
merged_comments = pd.merge(tweets_table_filtered,all_comments_df,left_on='KOL_Tweet_conversation_id',right_on='Comment_conversation_id' , how="inner")

In [51]:
merged_comments

,KOL_Tweet_Unnamed: 0.2,KOL_Tweet_Unnamed: 0,KOL_Tweet_Unnamed: 0.1,KOL_Tweet_author_id,KOL_Tweet_author_id_new,KOL_Tweet_conv_id_new,KOL_Tweet_conversation_id,KOL_Tweet_created_at,KOL_Tweet_entities.mentions,KOL_Tweet_id,...,Comment_users.created_at,Comment_users.description,Comment_users.id,Comment_users.name,Comment_users.public_metrics.followers_count,Comment_users.public_metrics.following_count,Comment_users.public_metrics.listed_count,Comment_users.public_metrics.tweet_count,Comment_users.username,Comment_users.verified
0,0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,7.401591e+17,...,2012-08-18T10:34:46.000Z,"interested in current affairs - world over,pla...",765477312,pongodhall,2277,4505,218,263066,pongodhall,False
1,0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,7.401591e+17,...,2012-08-18T10:34:46.000Z,"interested in current affairs - world over,pla...",765477312,pongodhall,2277,4505,218,263066,pongodhall,False
2,0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,7.401591e+17,...,2012-08-18T10:34:46.000Z,"interested in current affairs - world over,pla...",765477312,pongodhall,2277,4505,218,263066,pongodhall,False
3,0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,7.401591e+17,...,2012-08-18T10:34:46.000Z,"interested in current affairs - world over,pla...",765477312,pongodhall,2277,4505,218,263066,pongodhall,False
4,0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,7.401591e+17,...,2016-11-04T16:35:55.000Z,"Literacy, health, processes, nature, space, cu...",794578934511915008,Lorraine O'Mahoney 💙 #stopcancer,5575,6090,54,97942,Lomquiche,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5912,184,9583,496,2653613168,author_id: 2653613168,conv_id: 702594185942188033,702594185942188033,2016-02-24T20:41:22.000Z,NaN,7.025942e+17,...,2015-05-03T17:16:32.000Z,"Supporter of Liberty, Freedom and Democracy - ...",3231170470,Brian Tomkinson,2818,3616,17,243852,britom1947,False
5913,184,9583,496,2653613168,author_id: 2653613168,conv_id: 702594185942188033,702594185942188033,2016-02-24T20:41:22.000Z,NaN,7.025942e+17,...,2008-04-07T16:14:56.000Z,Career Commercial Radio/Music Management. Live...,14324310,John B Sheffield,934,1722,24,119916,jaybs,False
5914,184,9583,496,2653613168,author_id: 2653613168,conv_id: 702594185942188033,702594185942188033,2016-02-24T20:41:22.000Z,NaN,7.025942e+17,...,2009-06-24T18:46:43.000Z,"IT, SelloTape, String, the usual.\r\nI sit sl...",50394715,AdmV0rl0n,905,2685,31,43025,AdmV0rl0n,False
5915,184,9583,496,2653613168,author_id: 2653613168,conv_id: 702594185942188033,702594185942188033,2016-02-24T20:41:22.000Z,NaN,7.025942e+17,...,2013-12-07T06:22:28.000Z,Chair @ebbsfleetdc | Chair Square Roots RP | F...,2234063790,Simon Dudley,2640,500,53,13036,MrSimonDudley,False


In [92]:
merged_comments.columns

Index(['KOL_Tweet_Unnamed: 0', 'KOL_Tweet_Unnamed: 0.1', 'KOL_Tweet_author_id',
       'KOL_Tweet_conv_id_new', 'KOL_Tweet_conversation_id',
       'KOL_Tweet_created_at', 'KOL_Tweet_entities.mentions', 'KOL_Tweet_id',
       'KOL_Tweet_id_new', 'KOL_Tweet_public_metrics.like_count',
       'KOL_Tweet_public_metrics.quote_count',
       'KOL_Tweet_public_metrics.reply_count',
       'KOL_Tweet_public_metrics.retweet_count', 'KOL_Tweet_referenced_tweets',
       'KOL_Tweet_text', 'KOL_Tweet_users.created_at',
       'KOL_Tweet_users.description', 'KOL_Tweet_users.id',
       'KOL_Tweet_users.name',
       'KOL_Tweet_users.public_metrics.followers_count',
       'KOL_Tweet_users.public_metrics.following_count',
       'KOL_Tweet_users.public_metrics.listed_count',
       'KOL_Tweet_users.public_metrics.tweet_count',
       'KOL_Tweet_users.username', 'KOL_Tweet_users.verified',
       'KOL_Tweet_text_tokens', 'KOL_Tweet_created_at_date',
       'KOL_Tweet_is_in_special_date', 'KOL_Tweet_

In [16]:
final_table = merged_comments[['KOL_Tweet_author_id', 'KOL_Tweet_users.name', 'KOL_Tweet_conversation_id', 'KOL_Tweet_text', 'KOL_Tweet_text_tokens', 
'Comment_author_id', 'Comment_text',
       'Comment_users.created_at', 'Comment_users.description',
'Comment_users.public_metrics.following_count',
       'Comment_users.public_metrics.listed_count',
       'Comment_users.public_metrics.tweet_count', 'Comment_users.username',
       'Comment_users.verified' ]]

#final_table['KOL ID'] = final_table['KOL ID'].apply(lambda x: x.replace("author_id: ", ""))
#final_table['Commenter ID'] = final_table['Commenter ID'].apply(lambda x: x.replace("author_id: ", ""))

final_table

,KOL_Tweet_author_id,KOL_Tweet_users.name,KOL_Tweet_conversation_id,KOL_Tweet_text,KOL_Tweet_text_tokens,Comment_author_id,Comment_text,Comment_users.created_at,Comment_users.description,Comment_users.public_metrics.following_count,Comment_users.public_metrics.listed_count,Comment_users.public_metrics.tweet_count,Comment_users.username,Comment_users.verified
0,17487476,Michel Barnier,1164940747847667714,Goed om #Brexit te bespreken met@MinPres. TA =...,"['goed', 'om', 'brexit', 'te', 'bespreken', 'm...",26295559,@cswheatley @MichelBarnier You think the UK is...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,3973,41,100542,rovaz,False
1,17487476,Michel Barnier,1164940747847667714,Goed om #Brexit te bespreken met@MinPres. TA =...,"['goed', 'om', 'brexit', 'te', 'bespreken', 'm...",26295559,@YusufCilek3 @MichelBarnier If you have the ti...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,3973,41,100542,rovaz,False
2,17487476,Michel Barnier,1164940747847667714,Goed om #Brexit te bespreken met@MinPres. TA =...,"['goed', 'om', 'brexit', 'te', 'bespreken', 'm...",26295559,@patrick00054344 @MichelBarnier @MauriceMartin...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,3973,41,100542,rovaz,False
3,17487476,Michel Barnier,1164940747847667714,Goed om #Brexit te bespreken met@MinPres. TA =...,"['goed', 'om', 'brexit', 'te', 'bespreken', 'm...",26295559,@WTO_Rules @emeraldeagle2 @MichelBarnier Objec...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,3973,41,100542,rovaz,False
4,17487476,Michel Barnier,1164940747847667714,Goed om #Brexit te bespreken met@MinPres. TA =...,"['goed', 'om', 'brexit', 'te', 'bespreken', 'm...",26295559,@Stuartcassidy9 @MichelBarnier Ciao is actuall...,2009-03-24T18:50:13.000Z,Knows a bit about bytes. Invented social dista...,3973,41,100542,rovaz,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2653613168,Philip Hammond,702594185942188033,Britain's #EU deal has been registered at the ...,"['britain', 'eu', 'deal', 'registered', 'un', ...",50394715,@PHammondMP @Itwitius Its time to leave.,2009-06-24T18:46:43.000Z,"IT, SelloTape, String, the usual.\nI sit slig...",2685,31,43025,AdmV0rl0n,False
196,2653613168,Philip Hammond,702594185942188033,Britain's #EU deal has been registered at the ...,"['britain', 'eu', 'deal', 'registered', 'un', ...",2234063790,@PHammondMP Our Justice Secretary Michael Gove...,2013-12-07T06:22:28.000Z,Chair @ebbsfleetdc | Chair Square Roots RP | F...,500,53,13036,MrSimonDudley,False
197,2653613168,Philip Hammond,702594185942188033,Britain's #EU deal has been registered at the ...,"['britain', 'eu', 'deal', 'registered', 'un', ...",1119758522,@PHammondMP It's the deal that's not a deal. N...,2013-01-25T16:43:52.000Z,"They lie for a living, it's what they do",1000,0,50009,LiarMPs,False
198,2421073909,Martin Selmayr,859743329952706560,@nhaerting @VivianeRedingEU Und heute steht di...,"['nhaerting', 'vivianeredingeu', 'und', 'heute...",21111724,@MartinSelmayr @VivianeRedingEU Wir Berater si...,2009-02-17T18:00:24.000Z,Expect the unexpected. HÄRTING Rechtsanwälte s...,100,340,38133,nhaerting,False


In [ ]:
final_table.to_excel('final_table_with_comments.xlsx')